<a href="https://colab.research.google.com/github/josbex/HS-detection_in_social_media_posts/blob/master/Interpretation_of_BERT_using_captum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpretation of BertForSequenceClassification in captum

The orignial notebook this is based on can be found here:
https://colab.research.google.com/drive/1Lw3JTZio03VwPvSVFzLJmZ52oBRpo9ZM 

In [ ]:
# Install dependencies
!pip install transformers
!pip install captum

In [ ]:
import captum
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer
import torch
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import csv

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
def load_model(dir):
  output_dir = "./gdrive/My Drive/thesis/model/" + dir
  model = BertForSequenceClassification.from_pretrained(output_dir, output_attentions=True)
  tokenizer = BertTokenizer.from_pretrained(output_dir)
  # Copy the model to the GPU.
  model.to(device)
  model.eval()
  model.zero_grad()
  return model, tokenizer

In [ ]:
# load model and tokenizer
#model, tokenizer = load_model("/learn_rate_5/model_save") # model with 0.827 f1 score
model, tokenizer = load_model("new_training_round/best_model/model_save") # model with 0.819 f1 score

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
def predict(inputs):
    #print('model(inputs): ', model(inputs))
    return model(inputs)[0]

In [ ]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [ ]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [ ]:
def custom_forward(inputs):
    preds = predict(inputs)
    #return torch.softmax(preds, dim = 1)[:, 0] # for negative attribution, 
    return torch.softmax(preds, dim = 1)[:, 1] #<- for positive attribution

In [ ]:
lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

In [ ]:
def get_dataset(filename):
  df = pd.read_csv("/content/gdrive/My Drive/thesis/" + filename + ".tsv", sep="\t") 
  return df

In [ ]:
def get_test_tweet_by_index(index):
  df = get_dataset("dataset/test_data")
  tweets = df.tweet.values
  labels = df.label.values
  print("tweet: " + str(tweets[index]) + " label: " + str(labels[index]))
  return tweets[index], labels[index]

def get_prediction_type_indices(df):
  tp_indices = df.loc[df['Type'] == 'tp'].Index.values
  tn_indices = df.loc[df['Type'] == 'tn'].Index.values
  fp_indices = df.loc[df['Type'] == 'fp'].Index.values
  fn_indices = df.loc[df['Type'] == 'fn'].Index.values
  return tp_indices, tn_indices, fp_indices, fn_indices

In [ ]:
def tokenize_tweet(tweet):
  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(tweet, ref_token_id, sep_token_id, cls_token_id)
  token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  attention_mask = construct_attention_mask(input_ids)
  indices = input_ids[0].detach().tolist()
  all_tokens = tokenizer.convert_ids_to_tokens(indices)
  return input_ids, ref_input_ids, sep_id, token_type_ids, ref_token_type_ids, position_ids, ref_position_ids, attention_mask, indices, all_tokens

In [ ]:
df = get_dataset("model/predictions/pred_indices")
tp_indices, tn_indices, fp_indices, fn_indices = get_prediction_type_indices(df)

In [ ]:
def get_attributions(input_ids, ref_input_ids):
  attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    n_steps=7000,
                                    internal_batch_size=5,
                                    return_convergence_delta=True)
  return attributions, delta

In [ ]:
def print_tweet_with_prediction(tweet, score):
  print('Tweet: ', tweet)
  print('Sentiment: ' + str(torch.argmax(score[0]).cpu().numpy()) + \
        ', Probability positive: ' + str(torch.softmax(score, dim = 1)[0][1].cpu().detach().numpy()))

In [21]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [22]:
def save_vis(attributions_sum, score, true_label, tweet, all_tokens, delta):
  # storing couple samples in an array for visualization purposes
  score_vis = viz.VisualizationDataRecord(attributions_sum,
                                          torch.softmax(score, dim = 1)[0][0],
                                          torch.argmax(torch.softmax(score, dim = 1)[0]),
                                          true_label,
                                          tweet,
                                          attributions_sum.sum(),       
                                          all_tokens,
                                          delta)
  return score_vis

In [ ]:
print('fp: ' + str(len(fp_indices)))
print('tp: ' + str(len(tp_indices)))
print('fn: ' + str(len(fn_indices)))
print('tn: ' + str(len(tn_indices)))

fp: 57
tp: 172
fn: 68
tn: 563


In [24]:
def analyze_tweets(index_list, nr_of_tweets, title_of_prediction):
  vis = []
  for i in range(nr_of_tweets):
    tweet, true_label = get_test_tweet_by_index(index_list[i])
    input_ids, ref_input_ids, sep_id, token_type_ids, ref_token_type_ids, position_ids, ref_position_ids, attention_mask, indices, all_tokens = tokenize_tweet(tweet)
    attributions, delta = get_attributions(input_ids, ref_input_ids)
    attributions_sum = summarize_attributions(attributions)
    score = predict(input_ids)
    vis.append(save_vis(attributions_sum, score, true_label, tweet, all_tokens, delta))

  print('\033[1m', 'Visualization For Score ' + title_of_prediction, '\033[0m')
  for i in range(nr_of_tweets):
    viz.visualize_text([vis[i]])

index_lists = [fp_indices, tp_indices, fn_indices, tn_indices]
prediction_titles = ['False positive predictions', 'True positive predictions', 'False negative predictions', 'True negative predictions']

for i_list, title in zip(index_lists, prediction_titles):
  analyze_tweets(i_list, 15, title)

tweet: rap is a form of art ! used to express yourself freely . it does not gv the green light or excuse the behavior of acting like an animal ! she is not in the streets of the bx where violence is a way of living . elevate yourself boo and get on @user level for longevity ! queen label: 0
tweet: cons too the place for fed up conservatives ! ! ! label: 0
tweet: hiac damn matt hardy and randy orton put on one hell in a cell match ! ! woooo ! ! ! i hope he is okay ! ! label: 0
tweet: @user @user @user are you referring to how they lollygag with gun control as their kids get slaughtered in schools ? label: 0
tweet: sessions tells chicago : if you want more shootings , more death , listen to aclu , antifa , black lives matter url label: 0
tweet: are you fucking serious ? url label: 0
tweet: @user my only point of contention is this . he decided to keep this group which has top end talent and also decided to hire hot trash mike mccoy . wilks has not even been able to get his defense to lin

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,1 (0.06),cons too the place for fed up conservatives ! ! !,2.38,[CLS] con ##s too the place for fed up conservatives ! ! ! [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,1 (0.06),are you fucking serious ? url,1.08,[CLS] are you fucking serious ? ur ##l [SEP]


tweet: . . . what the fuck did he do this time ? label: 1
tweet: @user do you get the feeling he is kissing @user behind so he can humiliate him later ? label: 1
tweet: @user nigga ware da hits at label: 1
tweet: stop kavanaugh he is liar like the rest of the gop url label: 1
tweet: @user @user put delauro in a police lineup identifying the bag lady " - she would be picked everytime ! she has to be proof either conn voters are incompetent to vote or she is part of a vast voter fraud conspiracy ! no one votes for a woman that gross ! " label: 1
tweet: @ all family / friends , do not tell me bad shit that your bf / gf did to you just to go right back to them ! ! ! label: 1
tweet: ariana asesina ? is that serious ? ! holy shit , please your fucking assholes , do not blame someone for the death of other one . she is sad enough for today , do not you see ? it is not fault of none , he had an overdose and died . end . stop wanting someone to blame , fuckers . label: 1
tweet: @user how is she

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (0.06),. . . what the fuck did he do this time ?,1.89,[CLS] . . . what the fuck did he do this time ? [SEP]


...

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.95),"constitution day is revered by conservatives , hated by progressives / socialist / democrats that want to change it .",-0.20,"[CLS] constitution day is revered by conservatives , hated by progressive ##s / socialist / democrats that want to change it . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.99),tips to enhance audience connection on facebook url @user social media smm url,-1.86,[CLS] < number > tips to enhance audience connection on facebook ur ##l @ user social media sm ##m ur ##l [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.99),bigg boss tamil janani won the task . she is going to first final list,-0.90,[CLS] big ##g boss tamil jana ##ni won the task . she is going to first final list [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.98),conservatives - the party of low taxation tories tory url,-0.23,[CLS] conservatives - the party of low taxation tori ##es tory ur ##l [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.58),always smack url,1.26,[CLS] always smack ur ##l [SEP]
